In [4]:
import pandas as pd

df = pd.read_csv('data/referrals.csv')
df.head(5)

/tmp/ipykernel_11814/3977359288.py:3: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/referrals.csv')


,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
0,OPO1,OPO1_P320866,62.0,M,White / Caucasian,OPO1_H23456,False,NaN,Head Trauma,NaN,...,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OPO1,OPO1_P549364,14.0,F,White / Caucasian,OPO1_H11908,False,NaN,NaN,NaN,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OPO1,OPO1_P536997,55.0,M,White / Caucasian,OPO1_H23111,False,NaN,CVA/Stroke,ICH/Stroke,...,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OPO1,OPO1_P463285,48.0,F,Black / African American,OPO1_H26589,False,NaN,Anoxia,Cardiovascular,...,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OPO1,OPO1_P284978,80.0,F,White / Caucasian,OPO1_H5832,False,NaN,NaN,NaN,...,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
